# Chapter 7: Data Cleaning and Preparation

In [2]:
import numpy as np
import pandas as pd

## 7.1 Handling Missing Data

For numeric data, pandas uses `NaN` (Not a Number) to represent missing data.

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avacado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avacado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

Python's `None` is also treated as NA in object arrays.

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 7.1.1 Filtering Out Missing Data

`dropna` can be helpful to filter out missing data. On a Series, it returns the Series with only the non-null data and index values.

In [6]:
from numpy import nan as NA

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Is equivalent to index slicing on `isnull()`/`notnull()`.

In [8]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, `dropna` by default drops any row containing a missing value.

In [9]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                    [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` will only drop rows that are all NA.

In [11]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1`.

In [12]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep only rows containing a certain number of observations. Use the `thresh` argument.

In [24]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.717804,NaN,NaN
1,-1.328251,NaN,NaN
2,0.674920,NaN,0.326103
3,-1.078086,NaN,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


In [25]:
df.dropna()

,0,1,2
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


In [26]:
df.dropna(thresh=2)

,0,1,2
2,0.674920,NaN,0.326103
3,-1.078086,NaN,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


`.dropna(thresh=i)` only keeps rows that have at least "i"-non NA values.

### 7.1.2 Filling In Missing Data

For most purposes, use `fillna()` to fill in missing data. Calling `fillna(c)` with a constant replaces missing values with that value.

In [27]:
df.fillna(0)

,0,1,2
0,-0.717804,0.000000,0.000000
1,-1.328251,0.000000,0.000000
2,0.674920,0.000000,0.326103
3,-1.078086,0.000000,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


Calling `fillna` with a dict, you can use a different fill value for each column.

In [28]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.717804,0.500000,0.000000
1,-1.328251,0.500000,0.000000
2,0.674920,0.500000,0.326103
3,-1.078086,0.500000,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


`fillna` returns a new object, but you can modify in place.

In [29]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.717804,0.000000,0.000000
1,-1.328251,0.000000,0.000000
2,0.674920,0.000000,0.326103
3,-1.078086,0.000000,2.409767
4,-1.009092,-1.151225,-0.049840
5,2.323105,-0.150594,0.753602
6,1.759997,0.598949,-0.355756


Same interpolation methods available for reindexing can be used with `fillna`.

In [32]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.372242,-1.242153,1.326479
1,0.430150,-0.100254,-0.304015
2,-1.512890,NaN,1.737573
3,0.624016,NaN,0.069000
4,0.456371,NaN,NaN
5,-1.095843,NaN,NaN


In [33]:
df.fillna(method='ffill')

# 'ffill' takes the last observed value and fills the next valid NA

,0,1,2
0,0.372242,-1.242153,1.326479
1,0.430150,-0.100254,-0.304015
2,-1.512890,-0.100254,1.737573
3,0.624016,-0.100254,0.069000
4,0.456371,-0.100254,0.069000
5,-1.095843,-0.100254,0.069000


In [34]:
df.fillna(method='ffill', limit=2)

# 'limit=' limits how many values to fill

,0,1,2
0,0.372242,-1.242153,1.326479
1,0.430150,-0.100254,-0.304015
2,-1.512890,-0.100254,1.737573
3,0.624016,-0.100254,0.069000
4,0.456371,NaN,0.069000
5,-1.095843,NaN,0.069000


You can fill in values using the mean.

In [35]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 Data Transformations

### 7.2.1 Removing Duplicates

### 7.2.2 Transforming Data Using a Function or Mapping

### 7.2.3 Replacing Values

### 7.2.4 Renaming Axis Indexes

### 7.2.5 Discretization and Binning

### 7.2.6 Detecting and Filtering Outliers

### 7.2.7 Permutation and Random Sampling

### 7.2.8 Computing Indicator/Dummy Variables

## 7.3 String Manipulations

### 7.3.1 String Object Methods

### 7.3.2 Regular Expressions

### 7.3.3 Vectorized String Functions in pandas

## 7.4 Conclusion